<font size="+3"><strong>Data Wrangling with Mongodb</strong></font>

In [ ]:
from pprint import PrettyPrinter
import pandas as pd
from pymongo import MongoClient

In [ ]:
# Instantiate a PrettyPrinter
pp = PrettyPrinter (indent= 2)

# Prepare Data

## Connect

In [ ]:
# Create a client that connects to the database
client = MongoClient(host="localhost", port=27017)

## Explore

In [ ]:
# List of the available databases
pp.pprint(list(client.list_databases()))

[ {'empty': False, 'name': 'Air-Quality', 'sizeOnDisk': 33390592},
  {'empty': False, 'name': 'sample_airbnb', 'sizeOnDisk': 55078912},
  {'empty': False, 'name': 'sample_analytics', 'sizeOnDisk': 9572352},
  {'empty': False, 'name': 'sample_geospatial', 'sizeOnDisk': 1404928},
  {'empty': False, 'name': 'sample_restaurants', 'sizeOnDisk': 8994816},
  {'empty': False, 'name': 'sample_weatherdata', 'sizeOnDisk': 2875392},
  {'empty': False, 'name': 'admin', 'sizeOnDisk': 344064},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 24330530816}]


In [ ]:
# Assign the `air-quality` database to the variable db
db = client["Air-Quality"]

In [ ]:
# List of the collections available in `db`
for c in db.list_collections():
    print(c["name"])

beijing
lagos
mumbai
Delhi
dar-es-salaam


In [ ]:
# Assign the `beijing` collection in db to the variable name beijing
dar = db["dar-es-salaam"]

In [ ]:
# How many documents are in the `beijing` collection
dar.count_documents({})

198432

In [ ]:
# Retrieve one document from the `beijing` collection
result = dar.find_one({})
pp.pprint(result)

{ 'P2': 9.7,
  '_id': ObjectId('64f4f93a0102c916ca75eb72'),
  'metadata': { 'lat': -6.818,
                'lon': 39.285,
                'measurement': 'P2',
                'sensor_id': 29,
                'sensor_type': 'SDS011',
                'site': 11},
  'timestamp': datetime.datetime(2018, 1, 1, 0, 5, 3, 874000)}


In [ ]:
# How many sensor sites are included in the `beijing` collection
dar.distinct("metadata.site")

[11, 23]

In [ ]:
# How many readings there are for each site in the `beijing` collection
print("Documents from 11 Site:", dar.count_documents({"metadata.site":11}))
print("Documents from 23 Site:", dar.count_documents({"metadata.site": 23}))

Documents from 11 Site: 138412
Documents from 23 Site: 60020


In [ ]:
# How many readings there are for each site in the `beijing` collection
result = dar.aggregate(
       [
           {"$group":{"_id": "$metadata.site","count":{"$count":{}}}}
       ]
)
pp.pprint(list(result))

[{'_id': 23, 'count': 60020}, {'_id': 11, 'count': 138412}]


In [ ]:
# How many types of measurements have been taken in the `beijing` collection
dar.distinct("metadata.measurement")

['P1', 'P2', 'humidity', 'temperature']

In [ ]:
# Retrieve the PM 2.5 readings from all sites, limit 3
result = dar.find({"metadata.measurement": "P2"}). limit (3)
pp.pprint(list(result))

[ { 'P2': 9.7,
    '_id': ObjectId('64f4f93a0102c916ca75eb72'),
    'metadata': { 'lat': -6.818,
                  'lon': 39.285,
                  'measurement': 'P2',
                  'sensor_id': 29,
                  'sensor_type': 'SDS011',
                  'site': 11},
    'timestamp': datetime.datetime(2018, 1, 1, 0, 5, 3, 874000)},
  { 'P2': 8.97,
    '_id': ObjectId('64f4f93a0102c916ca75eba4'),
    'metadata': { 'lat': -6.818,
                  'lon': 39.285,
                  'measurement': 'P2',
                  'sensor_id': 29,
                  'sensor_type': 'SDS011',
                  'site': 11},
    'timestamp': datetime.datetime(2018, 1, 1, 2, 9, 53, 933000)},
  { 'P2': 9.37,
    '_id': ObjectId('64f4f93a0102c916ca75eb8d'),
    'metadata': { 'lat': -6.818,
                  'lon': 39.285,
                  'measurement': 'P2',
                  'sensor_id': 29,
                  'sensor_type': 'SDS011',
                  'site': 11},
    'timestamp': datetime.datet

In [ ]:
# Calculate how many readings there are for each measurements in site 11
result = dar.aggregate(
      [
          {"$match":{"metadata.site":11}},
          {"$group":{"_id": "$metadata.measurement","count":{"$count":{}}}}
      ]
)
pp.pprint(list(result))

[ {'_id': 'humidity', 'count': 17283},
  {'_id': 'temperature', 'count': 17283},
  {'_id': 'P1', 'count': 51923},
  {'_id': 'P2', 'count': 51923}]


In [ ]:
# Calculate how many readings there are for each measurements in site 23
result = dar.aggregate(
     [
          {"$match":{"metadata.site":23}},
          {"$group":{"_id": "$metadata.measurement","count":{"$count":{}}}}
     ]
)
pp.pprint(list(result))

[ {'_id': 'temperature', 'count': 15006},
  {'_id': 'P2', 'count': 15004},
  {'_id': 'humidity', 'count': 15006},
  {'_id': 'P1', 'count': 15004}]


## Import

In [ ]:
# Retrieve the `PM 2.5` readings from site 11
result = dar.find(
       {"metadata.site":11,"metadata.measurement":"P2"},
       projection= {"P2":1,"timestamp":1,"_id":0}
)
pp.pprint(result.next())

{'P2': 9.7, 'timestamp': datetime.datetime(2018, 1, 1, 0, 5, 3, 874000)}


In [ ]:
# Read records from result into the DataFrame `df`
df = pd.DataFrame(result).set_index("timestamp")
df.head()

,P2
timestamp,
2018-01-01 02:09:53.933,8.97
2018-01-01 01:12:29.094,9.37
2018-01-01 02:29:51.379,9.50
2018-01-01 02:57:24.358,7.87
2018-01-01 03:07:24.230,9.60
